In [ ]:
import uproot
import awkward as ak

import matplotlib.pylab as plt
import numpy as np

import hist
from hist import Hist

import time

import myPIDselector

import math



In [ ]:
eps = myPIDselector.PIDselector("e")
pps = myPIDselector.PIDselector("p")
pips = myPIDselector.PIDselector("pi")
Kps = myPIDselector.PIDselector("K")
mups = myPIDselector.PIDselector("mu")


In [ ]:
print(len(pps.selectors))

# There are 18 proton selectors, corresponding to the 18 proton selectors for BaBar release 24

pps.selectors

In [ ]:
pps.bits

In [ ]:
# ROOT
#'''
filename = 'bnv_plambda/SP-1005-LambdaVeryVeryLoose-Run3-R24a2-v03_COMBINED.root'

f = uproot.open(filename)

# This extracts the ROOT ntuple (TTree)
# From this we can extract awkward arrays
t = f['ntp1']

print(type(t))
#'''



In [ ]:
n = 1

pe = t['penergy'].array()[n]
trke = t['TRKenergy'].array()[n]

pbit = t['pSelectorsMap'].array()[n]
ptrkidx = t['pTrkIdx'].array()[n]


print(pbit)
print(trke)
print()
print(pe)
print(ptrkidx)
print()

#pps.SetBits(pbit)

#binrep = np.binary_repr(pbit)

#? np.binary_repr

binary_repr_vec = np.vectorize(np.binary_repr)
binrep = binary_repr_vec(pbit, width=32)

print(binrep)
print()
#tempbits = np.array(list(binrep[::-1])).astype(int)
tempbits = np.array(binrep).astype(int)

print(tempbits)
print()


pps.SetBits(pbit[0])
print(pps.IsBitSet(15))
print(pps.IsSelectorSet("LooseKMProtonSelection"))
print(pps.IsSelectorSet("TightKMProtonSelection"))
print(pps.IsSelectorSet("VeryTightKMProtonSelection"))
print(pps.IsSelectorSet("SuperTightKMProtonSelection"))

In [ ]:
# Trying to get ak working

n = 1

pe = t['penergy'].array()[n]
trke = t['TRKenergy'].array()[n]

pbit = t['pSelectorsMap'].array()[n]
ptrkidx = t['pTrkIdx'].array()[n]


print(pbit)
print(trke)
print()
print(pe)
print(ptrkidx)
print()

x = pbit[ptrkidx]
print("pbit[ptrkidx]")
print(x)
print()

#pps.SetBits(pbit)

#binrep = np.binary_repr(pbit)

#? np.binary_repr

binary_repr_vec = np.vectorize(np.binary_repr)
binrep = binary_repr_vec(x, width=32)

print(binrep)
print()
#tempbits = np.array(list(binrep[::-1])).astype(int)
tempbits = np.array(binrep).astype(int)

print(tempbits)
print()


#pps.SetBits(pbit[0])
#print(pps.IsBitSet(15))
#print(pps.IsSelectorSet("LooseKMProtonSelection"))
#print(pps.IsSelectorSet("TightKMProtonSelection"))
#print(pps.IsSelectorSet("VeryTightKMProtonSelection"))
#print(pps.IsSelectorSet("SuperTightKMProtonSelection"))

In [ ]:
# Trying to get ak working

# Loops are slow!

allbits = []

#nevents = len(t['penergy'].array())
nevents = 10

for n in range(nevents):
    #n = 1

    if n%10==0:
        print(n)
        
    pe = t['penergy'].array()[n]
    trke = t['TRKenergy'].array()[n]
    
    pbit = t['pSelectorsMap'].array()[n]
    ptrkidx = t['pTrkIdx'].array()[n]
    
    
    #print(pbit)
    #print(trke)
    #print()
    #print(pe)
    #print(ptrkidx)
    #print()
    
    x = pbit[ptrkidx]
    #print("pbit[ptrkidx]")
    #print(x)
    #print()
    
    #pps.SetBits(pbit)
    
    #binrep = np.binary_repr(pbit)
    
    #? np.binary_repr
    
    binary_repr_vec = np.vectorize(np.binary_repr)
    binrep = binary_repr_vec(x, width=32)
    
    #print(binrep)
    #print()
    #tempbits = np.array(list(binrep[::-1])).astype(int)
    tempbits = np.array(binrep).astype(int)
    
    #print(tempbits)
    #print()

    allbits.append(tempbits)

allbits = ak.Array(allbits)

allbits

#pps.SetBits(pbit[0])
#print(pps.IsBitSet(15))
#print(pps.IsSelectorSet("LooseKMProtonSelection"))
#print(pps.IsSelectorSet("TightKMProtonSelection"))
#print(pps.IsSelectorSet("VeryTightKMProtonSelection"))
#print(pps.IsSelectorSet("SuperTightKMProtonSelection"))

In [ ]:
allbits[2]

In [ ]:
nevents = len(t['penergy'].array())
print(f"nevents: {nevents}")

n = nevents

pe = t['penergy'].array()[0:n]
trke = t['TRKenergy'].array()[0:n]

pbit = t['pSelectorsMap'].array()[0:n]
ptrkidx = t['pTrkIdx'].array()[0:n]


print(pbit)
print(trke)
print()
print(pe)
print(ptrkidx)
print()

binary_repr_vec = np.vectorize(np.binary_repr)

counts = ak.num(pbit)
binrep = binary_repr_vec(ak.flatten(pbit), width=32)

print(binrep)
print()
#tempbits = np.array(list(binrep[::-1])).astype(int)
tempbits = np.array(binrep).astype(int)

print(tempbits)
print()

newtempbits = ak.unflatten(tempbits,counts)
print(newtempbits)

place = int(math.pow(10,12))
print(place)

test = newtempbits // place % 10
print(test)

'''
pps.SetBits(pbit[0])
print(pps.IsBitSet(15))
print(pps.IsSelectorSet("LooseKMProtonSelection"))
print(pps.IsSelectorSet("TightKMProtonSelection"))
print(pps.IsSelectorSet("VeryTightKMProtonSelection"))
print(pps.IsSelectorSet("SuperTightKMProtonSelection"))
'''
;

In [ ]:
print(pps.selectors)

In [ ]:
pps.selectors.index('TightKMProtonSelection')

In [ ]:
def calculate_bits_for_PID_selector(trkidx, trk_selector_map, verbose=0):
    
    bits = None

    # Grab the tracks that map on to the particle/collection we are interested in 
    subset_of_trk_selector_map = trk_selector_map[trkidx]
    if verbose:
        print("values in the subset of the trk selector map")
        print(subset_of_trk_selector_map)
        print()
        
    # We need this to convert the numbers in the selector to binary
    binary_repr_vec = np.vectorize(np.binary_repr)

    # Grab the number of entries in each so we can unflatten this later
    counts = ak.num(subset_of_trk_selector_map)
    
    # Now get the binary representation (as a string) for the flattened subset
    binrep = binary_repr_vec(ak.flatten(subset_of_trk_selector_map), width=32)

    if verbose:
        print("binary representation of selector map")
        print(binrep)
        print()

    # Convert the string to integers
    tempbits = np.array(binrep).astype(int)
    bits = ak.unflatten(tempbits,counts)

    if verbose:
        print("unflattened integer representation of selectors as binary (int)")
        print(tempbits)
        print()
        print("flattened integer representation of selectors as binary (int)")
        print(bits)
        print()

    return bits

##########################################################################
print("protons ====================")

trkidx = t['pTrkIdx'].array()
trk_selector_map = t['pSelectorsMap'].array()

print(trkidx[0:3])
print(trk_selector_map[0:3])

pbits = calculate_bits_for_PID_selector(trkidx, trk_selector_map, verbose=1)
print(pbits)
print(len(pbits))
print()

print("pions ====================")

trkidx = t['piTrkIdx'].array()
trk_selector_map = t['piSelectorsMap'].array()

print(trkidx[0:3])
print(trk_selector_map[0:3])

pibits = calculate_bits_for_PID_selector(trkidx, trk_selector_map, verbose=1)
print(pibits)
print(len(pibits))
print()



In [ ]:
def mask_PID_selection(bits, selector):

    bit_to_look_for = pps.selectors.index(selector)
    
    place = int(math.pow(10,bit_to_look_for))
    #print(place)

    mask = bits // place % 10

    mask_bool = ak.values_astype(mask,bool)

    return mask_bool
    

#selector = 'LooseKMProtonSelection'
selector = 'TightKMProtonSelection'
#selector = 'VeryLooseLHProtonSelection'

mask_bool = mask_PID_selection(pbits, selector)

print(pbits)
#mask_bool = ak.values_astype(mask,bool)
#print(mask)
#print(mask_bool)

print()
print(t['penergy'].array()[mask_bool])

In [ ]:
# Figuring out how to use B and Lambda0 PID masks
mass = t['Lambda0_unc_Mass'].array()#[:,0]

print("mass")
print(mass)
print()

d1idx = t['Lambda0d1Idx'].array()#[:,0]
d2idx = t['Lambda0d2Idx'].array()#[:,0]
d1lund = t['Lambda0d1Lund'].array()#[:,0]
d2lund = t['Lambda0d2Lund'].array()#[:,0]

print(d1idx)
print(d2idx)
print(d1lund)
print(d2lund)

print("\nTrying to get the bits of the daughters...")
print(pbits)
print(pbits[d1idx])
print()

selector = 'LooseKMProtonSelection'
mask_bool = mask_PID_selection(pbits[d1idx], selector)
print(mask_bool)
passing_pbits = pbits[d1idx][mask_bool]
print("\nTrying to get the masked bits of the daughters...")
print(pbits[d1idx][mask_bool])
print()


#pibits[


plt.figure(figsize=(12,5));
plt.subplot(1,2,1)
plt.hist(ak.flatten(mass),bins=100)
plt.hist(ak.flatten(mass[mask_bool]),bins=100)

print(len(ak.flatten(mass)))
print(len(ak.flatten(mass[mask_bool])))

plt.subplot(1,2,2)

Bmass = t['BMass'].array()#[:,0]
Bmass = t['BpostFitMes'].array()#[:,0]

plt.hist(ak.flatten(Bmass),bins=100, range=(5,5.3))
plt.hist(ak.flatten(Bmass[mask_bool]),bins=100, range=(5,5.3))

print(len(ak.flatten(Bmass)))
print(len(ak.flatten(Bmass[mask_bool])))

;

In [ ]:
lamd1idx = t['Lambda0d1Idx'].array()#[:,0]
lamd1lund = t['Lambda0d1Lund'].array()#[:,0]

print(lamd1idx)
print(lamd1lund)

print("\nTrying to get the bits of the daughters...")
print(pbits[lamd1idx])
print()

selector = 'LooseKMProtonSelection'
lam_mask_bool = mask_PID_selection(pbits[lamd1idx], selector)
print(lam_mask_bool)
lam_passing_pbits = pbits[lamd1idx][lam_mask_bool]
print("\nTrying to get the masked bits of the daughters...")
print(pbits[lamd1idx][lam_mask_bool])
print()


# Figuring out how to use B and Lambda0 PID masks
d1idx = t['Bd1Idx'].array()#[:,0]
d2idx = t['Bd2Idx'].array()#[:,0]
d1lund = t['Bd1Lund'].array()#[:,0]
d2lund = t['Bd2Lund'].array()#[:,0]

# d1 Lambda0
# d2 proton

print(d1idx)
print(d2idx)
print(d1lund)
print(d2lund)

print("\nTrying to get the bits of the daughters...")
print(pbits[d2idx])
print()

selector = 'LooseKMProtonSelection'
mask_bool = mask_PID_selection(pbits[d2idx], selector)
print(mask_bool)
passing_pbits = pbits[d2idx][mask_bool]
print("\nTrying to get the masked bits of the daughters...")
print(pbits[d2idx][mask_bool])
print()


#pibits[
Bmass1 = t['BMass'].array()#[:,0]
Bmass2 = t['BpostFitMes'].array()#[:,0]


plt.figure(figsize=(12,5));
plt.subplot(1,2,1)
plt.hist(ak.flatten(Bmass1),bins=100, range=(4,6))
plt.hist(ak.flatten(Bmass1[mask_bool]),bins=100, range=(4,6))
plt.hist(ak.flatten(Bmass1[mask_bool & lam_mask_bool]),bins=100, range=(4,6))

print(len(ak.flatten(Bmass)))
print(len(ak.flatten(Bmass[mask_bool])))
print(len(ak.flatten(Bmass[mask_bool & lam_mask_bool])))

plt.yscale('log')

plt.subplot(1,2,2)


plt.hist(ak.flatten(Bmass2),bins=100, range=(5,5.3))
plt.hist(ak.flatten(Bmass2[mask_bool]),bins=100, range=(5,5.3))
plt.hist(ak.flatten(Bmass2[mask_bool & lam_mask_bool]),bins=100, range=(5,5.3))

print(len(ak.flatten(Bmass2)))
print(len(ak.flatten(Bmass2[mask_bool])))
print(len(ak.flatten(Bmass2[mask_bool & lam_mask_bool])))

plt.yscale('log')

;

In [ ]:
n = 1

lamd1idx = t['Lambda0d1Idx'].array()[n]
lamd1lund = t['Lambda0d1Lund'].array()[n]
print(lamd1idx)
print(lamd1lund)
print()


pe = t['penergy'].array()[n]
trke = t['TRKenergy'].array()[n]

pbit = t['pSelectorsMap'].array()[n]
ptrkidx = t['pTrkIdx'].array()[n]


print(pbit)
print(trke)
print()
print(pe)
print(ptrkidx)
print()

binary_repr_vec = np.vectorize(np.binary_repr)
binrep = binary_repr_vec(pbit, width=32)

print(binrep)
print()
#tempbits = np.array(list(binrep[::-1])).astype(int)
tempbits = np.array(binrep).astype(int)

print(tempbits)
print()


pps.SetBits(pbit[0])
print(pps.IsBitSet(15))
print(pps.IsSelectorSet("LooseKMProtonSelection"))
print(pps.IsSelectorSet("TightKMProtonSelection"))
print(pps.IsSelectorSet("VeryTightKMProtonSelection"))
print(pps.IsSelectorSet("SuperTightKMProtonSelection"))

In [ ]:
test = ak.Array([[1000010, 100], [1000], [1100, 11111111]])

test

In [ ]:
count = ak.num(test)
x = ak.flatten(test)

x

In [ ]:
x%1000

In [ ]:
print(x)
x//1000 % 10


# Try to get working with parquet

In [ ]:
# Parquet

# At Siena
topdir = '/mnt/qnap/babar_data/bnv_plambda'

# At home (bellis)
#topdir = '/home/bellis/babar_data/bnv_plambda'


#filename = f'{topdir}/Background_SP_modes_Only_Run_1.parquet'
#filename = f'{topdir}/Background_SP_modes_Only_Run_1.parquet'
filename = f'{topdir}/Background_and_signal_SP_modes_All_runs.parquet'

# Signal
#filename = f'{topdir}/Signal_SP_mode.parquet'

start = time.time()

data = ak.from_parquet(filename)

print(f"Took {time.time() - start} s")

print(type(data))

In [ ]:
mask = data['spmode']=='1235'
x = data[mask]['nB']

print(ak.sum(x))
print(len(x))

In [ ]:
eps = myPIDselector.PIDselector("e")
pps = myPIDselector.PIDselector("p")
pips = myPIDselector.PIDselector("pi")
Kps = myPIDselector.PIDselector("K")
mups = myPIDselector.PIDselector("mu")


print(len(pps.selectors))

pps.selectors

In [ ]:
def calculate_bits_for_PID_selector(trkidx, trk_selector_map, verbose=0):
    
    bits = None

    # Grab the tracks that map on to the particle/collection we are interested in 
    if trkidx is None:
        subset_of_trk_selector_map = trk_selector_map
    else:
        subset_of_trk_selector_map = trk_selector_map[trkidx]
    if verbose:
        print("values in the subset of the trk selector map")
        print(subset_of_trk_selector_map)
        print()
        
    # We need this to convert the numbers in the selector to binary
    binary_repr_vec = np.vectorize(np.binary_repr)

    # Grab the number of entries in each so we can unflatten this later
    counts = ak.num(subset_of_trk_selector_map)
    
    # Now get the binary representation (as a string) for the flattened subset
    binrep = binary_repr_vec(ak.flatten(subset_of_trk_selector_map), width=32)

    if verbose:
        print("binary representation of selector map")
        print(binrep)
        print()

    # Convert the string to integers
    tempbits = np.array(binrep).astype(int)
    bits = ak.unflatten(tempbits,counts)

    if verbose:
        print("unflattened integer representation of selectors as binary (int)")
        print(tempbits)
        print()
        print("flattened integer representation of selectors as binary (int)")
        print(bits)
        print()

    return bits

##########################################################################
print("protons ====================")

trkidx = data['pTrkIdx']
trk_selector_map = data['pSelectorsMap']

print(trkidx[0:3])
print(trk_selector_map[0:3])

pbits = calculate_bits_for_PID_selector(trkidx, trk_selector_map, verbose=1)
print(pbits)
print(len(pbits))
print()

print("pions ====================")

trkidx = data['piTrkIdx']
trk_selector_map = data['piSelectorsMap']

print(trkidx[0:3])
print(trk_selector_map[0:3])

pibits = calculate_bits_for_PID_selector(trkidx, trk_selector_map, verbose=1)
print(pibits)
print(len(pibits))
print()



In [ ]:
def mask_PID_selection(bits, selector):

    bit_to_look_for = pps.selectors.index(selector)
    
    place = int(math.pow(10,bit_to_look_for))
    #print(place)

    mask = bits // place % 10

    mask_bool = ak.values_astype(mask,bool)

    return mask_bool
    

#selector = 'LooseKMProtonSelection'
selector = 'TightKMProtonSelection'
#selector = 'VeryLooseLHProtonSelection'

pbits = calculate_bits_for_PID_selector(trkidx, trk_selector_map, verbose=0)

mask_bool = mask_PID_selection(pbits, selector)

print(pbits)
#mask_bool = ak.values_astype(mask,bool)
#print(mask)
#print(mask_bool)

print()
print(data['penergy'][mask_bool])

## Using the mask in analysis

In [ ]:
# Figuring out how to use B and Lambda0 PID masks
mass = data['Lambda0_unc_Mass']#[:,0]

print("mass")
print(mass)
print()

d1idx = data['Lambda0d1Idx']#[:,0]
d2idx = data['Lambda0d2Idx']#[:,0]
d1lund = data['Lambda0d1Lund']#[:,0]
d2lund = data['Lambda0d2Lund']#[:,0]

print("d1idx")
print(d1idx)
print("d2idx")
print(d2idx)

print("d1lund")
print(d1lund)
print("d2lund")
print(d2lund)

print("\nTrying to get the bits of the daughters...")
trkidx = data['pTrkIdx']
trk_selector_map = data['pSelectorsMap']
print()

print("trkidx")
print(trkidx)
print("trk_selector_map")
print(trk_selector_map)

pbits = calculate_bits_for_PID_selector(trkidx, trk_selector_map, verbose=1)
print(pbits)
print(pbits[d1idx])
print()

selector = 'LooseKMProtonSelection'
print(f"Now trying to create a mask with {selector}")

mask_bool = mask_PID_selection(pbits[d1idx], selector)
print(mask_bool)
print()
passing_pbits = pbits[d1idx][mask_bool]
print("\nTrying to get the masked bits of the daughters...")
print(pbits[d1idx][mask_bool])
print()


#pibits[


plt.figure(figsize=(12,5));
plt.subplot(1,2,1)
plt.hist(ak.flatten(mass),bins=100)
plt.hist(ak.flatten(mass[mask_bool]),bins=100)

print(len(ak.flatten(mass)))
print(len(ak.flatten(mass[mask_bool])))

plt.subplot(1,2,2)

Bmass = data['BMass']#[:,0]
Bmass = data['BpostFitMes']#[:,0]

plt.hist(ak.flatten(Bmass),bins=100, range=(5,5.3))
plt.hist(ak.flatten(Bmass[mask_bool]),bins=100, range=(5,5.3))

print(len(ak.flatten(Bmass)))
print(len(ak.flatten(Bmass[mask_bool])))

;

# Testing some cuts

In [ ]:
%load_ext autoreload
%autoreload 2

import uproot
import awkward as ak

import matplotlib.pylab as plt
import numpy as np

import time

import pandas as pd

from hist import Hist

import babar_analysis_tools as bat

import myPIDselector

In [ ]:
start = time.time()


# At Siena
#topdir = '/mnt/qnap/babar_data/bnv_plambda'

# At home (Bellis)
topdir = '/home/bellis/babar_data/bnv_plambda'


# Read in SP
#filename = f'{topdir}/Background_SP_modes_Only_Run_1.parquet'
filename = f'{topdir}/Background_and_signal_SP_modes_Only_Run_1.parquet'
#filename = f'{topdir}/Background_and_signal_SP_modes_All_runs.parquet'

start = time.time()
ak_sp = ak.from_parquet(filename)
print(f"Took {time.time() - start} s")


# Read in data
filename = f'{topdir}/Data_Only_Run_1_BLINDED.parquet'
#filename = f'{topdir}/Data_All_runs_BLINDED.parquet'
start = time.time()
ak_data = ak.from_parquet(filename)
print(f"Took {time.time() - start} s")



In [ ]:
spmask = ak_sp['spmode']=='-999'

ak_sp = ak_sp[spmask]

In [ ]:
q = (ak_sp['Bd2Lund'][:,0])/np.abs(ak_sp['Bd2Lund'][:,0])
q

In [ ]:
eps = myPIDselector.PIDselector("e")
pps = myPIDselector.PIDselector("p")
pips = myPIDselector.PIDselector("pi")
Kps = myPIDselector.PIDselector("K")
mups = myPIDselector.PIDselector("mu")

In [ ]:
lamd1idx = ak_sp['Lambda0d1Idx']
lamd1Lund = ak_sp['Lambda0d1Lund']

print(f'lamd1idx\n{lamd1idx}')
print(f'lamd1Lund\n{lamd1Lund}')


d2idx = ak_sp['Bd2Idx']
d2Lund = ak_sp['Bd2Lund']

print(f'd2idx\n{d2idx}')
print(f'd2Lund\n{d2Lund}')


trk_selector_map = ak_sp['pSelectorsMap']
print()

print("trk_selector_map")
print(trk_selector_map)

# Do this for all the tracks
pbits = calculate_bits_for_PID_selector(None, trk_selector_map, verbose=1)
print(f'pbits:\n{pbits}')
print(f'pbits[d2idx]:\n{pbits[d2idx]}')
print()

selector = 'LooseKMProtonSelection'
print(f"Now trying to create a mask with {selector}")

mask_bool = mask_PID_selection(pbits[d2idx], selector)
print(f'mask_book: {mask_bool}')
print()
passing_pbits = pbits[d2idx][mask_bool]
print("\nMasked bits of the daughters...")
print(f'{passing_pbits}')
print()

print("\nMasked bits of the not daughters...")
passing_pbits_not_daughter = pbits[~d2idx][mask_bool]
print(f'{passing_pbits_not_daughter}')
print()

In [ ]:
ak_sp.TRKLund

In [ ]:
lund = ak_sp.TRKLund

n = ak.num(lund)

trkq = ak.unflatten((ak.flatten(lund)/np.abs(ak.flatten(lund))),n)

trkq

https://stackoverflow.com/questions/78657390/accessing-elements-of-an-awkward-array-that-are-not-a-passed-in-index

In [ ]:
def indices_to_booleans(indices, array_to_slice):
    whole_set, in_set = ak.unzip(ak.cartesian([
        ak.local_index(array_to_slice), indices
    ], nested=True))
    return ak.any(whole_set == in_set, axis=-1)

In [ ]:
idx = indices_to_booleans(d2idx, trkq)

In [ ]:
trkq[idx]

In [ ]:
trkq[~idx]

In [ ]:
trkq

In [ ]:
pbits[~idx]

In [ ]:
n = ak.num(ak_sp.TRKLund)
trkq = ak.flatten(ak_sp.TRKLund)/np.abs(ak.flatten(ak_sp.TRKLund))
trkq = ak.unflatten(trkq,n)
trkq

In [ ]:
d2idx

In [ ]:
trkq[~d2idx]

In [ ]:
d2idx[:,0]

In [ ]:
mask_idx = ak.ones_like(trkq, dtype=bool)
for m in mask_idx[0:10]:
    print(m)
print()

m = mask_idx[d2idx]# = False
print(m)


'''
for m in mask_idx[0:10]:
    print(m)
print()
'''
;

In [ ]:
#mask_idx = mask_idx[d2idx[:,0]] = False

#mask_idx

In [ ]:
#mask_idx[d2idx] = False

In [ ]:
arr = ak.Array([[1, 2, 3], [4,5],[7], [8, 9,10]])
arr

In [ ]:
idx = ak.Array([[0,0], [0], [0], [0]])
idx

In [ ]:
arr[idx]
#arr

# Example

In [ ]:
jet_lepton_indices = ak.Array([[0, 2], [1], [2,3]])
print(f'jet_lepton_indices\n{jet_lepton_indices}\n')

lepton_flags = ak.Array([[0, 10, 20, 30], [0, 10, 20, 30], [0, 10, 20, 30, 40]])
print(f'lepton_flags\n{lepton_flags}\n')

lepton_flags[jet_lepton_indices]

In [ ]:
ak.local_index(lepton_flags)

In [ ]:
lepton_flags

In [ ]:
array = ak.Array(
    [
        [
            [0.0, 1.1, 2.2],
            [3.3, 4.4, 5.5, 6.6],
            [7.7],
        ],
        [],
        [
            [8.8, 9.9, 10.10, 11.11, 12.12],
        ],
    ]
)
array

In [ ]:
ak.local_index(array)

In [ ]:
index = ak.Array(
    [
        [[], [0], [0]],
        [],
        [[2, 3, 4]],
    ]
)

In [ ]:
array[index]